In [7]:
%%writefile ddp.py

import subprocess
import sys

subprocess.run(
    [sys.executable, "-m", "pip", "install", "evaluate", "-q"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

import os
import numpy as np
import torch
import torch.optim as optim
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from tqdm import tqdm
import json

from transformers import (
    AutoModelForTokenClassification, 
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments, 
    Trainer,
    get_linear_schedule_with_warmup
)
from datasets import load_dataset
import evaluate

# ===== 数据处理模块 =====
def load_and_prepare_data():
    """加载并预处理数据集"""
    # 加载数据集
    ds = load_dataset('nlhappy/CLUE-NER')
    
    # 定义实体类别和标签
    entities = ['O'] + list({
        'movie', 'name', 'game', 'address', 'position', 
        'company', 'scene', 'book', 'organization', 'government'
    })
    
    tags = ['O']
    for entity in entities[1:]:
        tags.append('B-' + entity.upper())
        tags.append('I-' + entity.upper())
    
    entity_index = {entity: i for i, entity in enumerate(entities)}
    
    return ds, tags, entity_index

def process_entity_tags(ds, entity_index):
    """处理实体标签"""
    def entity_tags_proc(item):
        text_len = len(item['text'])
        tags = [0] * text_len
        entities = item['ents']
        for ent in entities:
            indices = ent['indices']
            label = ent['label']
            tags[indices[0]] = entity_index[label] * 2 - 1
            for idx in indices[1:]:
                tags[idx] = entity_index[label] * 2
        return {'ent_tag': tags}
    
    return ds.map(entity_tags_proc)

def tokenize_data(ds, tokenizer):
    """分词处理"""
    def data_input_proc(item):
        batch_texts = [list(text) for text in item['text']]
        input_data = tokenizer(
            batch_texts, 
            truncation=True, 
            add_special_tokens=False, 
            max_length=512,
            is_split_into_words=True, 
            padding='max_length'
        )
        input_data['labels'] = [tag + [0] * (512 - len(tag)) for tag in item['ent_tag']]
        return input_data
    
    return ds.map(data_input_proc, batched=True)

# ===== 模型训练模块 =====
def create_model_and_optimizer(tags, learning_rates=None):
    """创建模型和优化器"""
    if learning_rates is None:
        learning_rates = {'bert': 1e-5, 'classifier': 1e-3}
    
    # 创建标签映射
    id2lbl = {i: tag for i, tag in enumerate(tags)}
    lbl2id = {tag: i for i, tag in enumerate(tags)}
    
    # 加载模型
    model = AutoModelForTokenClassification.from_pretrained(
        'google-bert/bert-base-chinese',
        num_labels=len(tags),
        id2label=id2lbl,
        label2id=lbl2id
    )
    
    # 参数分组
    param_optimizer = list(model.named_parameters())
    bert_params = [params for name, params in param_optimizer if 'bert' in name]
    classifier_params = [params for name, params in param_optimizer if 'bert' not in name]
    
    param_groups = [
        {'params': bert_params, 'lr': learning_rates['bert']},
        {'params': classifier_params, 'weight_decay': 0.1, 'lr': learning_rates['classifier']}
    ]
    
    optimizer = optim.AdamW(param_groups)
    return model, optimizer

def setup_distributed(rank, world_size):
    """设置分布式训练环境"""
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)

def cleanup_distributed():
    """清理分布式训练环境"""
    dist.destroy_process_group()

def train_ddp(rank, world_size, train_dataset, tags, epochs=3, batch_size=16, use_amp=False):
    """分布式DDP训练函数"""
    print(f"Running DDP on rank {rank}")
    setup_distributed(rank, world_size)
    
    # 创建模型和优化器
    model, optimizer = create_model_and_optimizer(tags)
    model = model.to(rank)
    model = DDP(model, device_ids=[rank])
    
    # 创建分布式数据加载器
    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size,
        sampler=train_sampler,
        pin_memory=True
    )
    
    # 学习率调度器
    train_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=100,
        num_training_steps=train_steps
    )
    
    # 混合精度训练
    scaler = torch.cuda.amp.GradScaler() if use_amp else None
    
    # 训练循环
    for epoch in range(epochs):
        train_sampler.set_epoch(epoch)
        model.train()
        
        if rank == 0:
            pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
        else:
            pbar = train_loader
            
        total_loss = 0
        for batch_idx, items in enumerate(pbar):
            items = {k: v.to(rank) for k, v in items.items()}
            optimizer.zero_grad()
            
            if use_amp:
                with torch.cuda.amp.autocast():
                    outputs = model(**items)
                loss = outputs.loss
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(**items)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
            
            scheduler.step()
            total_loss += loss.item()
            
            if rank == 0 and isinstance(pbar, tqdm):
                pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        
        if rank == 0:
            avg_loss = total_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")
    
    # 保存模型 (只在rank 0保存)
    if rank == 0:
        save_model(model.module, tags, "/kaggle/working/ner_ddp_model")
        print("DDP训练完成，模型已保存！")
    
    cleanup_distributed()

def save_model(model, tags, save_dir):
    """保存模型和配置"""
    os.makedirs(save_dir, exist_ok=True)
    
    # 保存模型权重
    torch.save(model.state_dict(), os.path.join(save_dir, "/kaggle/working/model_weights.pth"))
    
    # 保存标签映射
    label_config = {
        'tags': tags,
        'id2label': {i: tag for i, tag in enumerate(tags)},
        'label2id': {tag: i for i, tag in enumerate(tags)}
    }
    with open(os.path.join(save_dir, "/kaggle/working/label_config.json"), 'w', encoding='utf-8') as f:
        json.dump(label_config, f, indent=2, ensure_ascii=False)
    
    print(f"模型已保存到: {save_dir}")

def load_model(save_dir):
    """加载保存的模型"""
    # 加载标签配置
    with open(os.path.join(save_dir, "/kaggle/working/label_config.json"), 'r', encoding='utf-8') as f:
        label_config = json.load(f)
    
    tags = label_config['tags']
    id2label = label_config['id2label']
    label2id = label_config['label2id']
    
    # 创建模型
    model = AutoModelForTokenClassification.from_pretrained(
        'google-bert/bert-base-chinese',
        num_labels=len(tags),
        id2label=id2label,
        label2id=label2id
    )
    
    # 加载权重
    model.load_state_dict(torch.load(os.path.join(save_dir, "/kaggle/working/model_weights.pth")))
    
    # 加载分词器
    tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
    
    return model, tokenizer, tags

def predict_entities(model, tokenizer, tags, text):
    """推理函数"""
    model.eval()
    device = next(model.parameters()).device
    
    # 处理输入文本
    chars = list(text)
    inputs = tokenizer(
        chars,
        return_tensors='pt',
        is_split_into_words=True,
        padding=True,
        truncation=True,
        max_length=512
    )
    
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 预测
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    
    # 解析结果
    predicted_labels = [tags[pred.item()] for pred in predictions[0]]
    
    # 提取实体
    entities = []
    current_entity = None
    
    for i, (char, label) in enumerate(zip(chars, predicted_labels)):
        if label.startswith('B-'):
            if current_entity:
                entities.append(current_entity)
            current_entity = {
                'text': char,
                'label': label[2:],
                'start': i,
                'end': i + 1
            }
        elif label.startswith('I-') and current_entity and label[2:] == current_entity['label']:
            current_entity['text'] += char
            current_entity['end'] = i + 1
        else:
            if current_entity:
                entities.append(current_entity)
                current_entity = None
    
    if current_entity:
        entities.append(current_entity)
    
    return entities

def compute_metrics(eval_pred):
    """计算评估指标"""
    seqeval = evaluate.load('seqeval')
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)
    
    # 获取标签映射
    _, tags, _ = load_and_prepare_data()
    
    # 准备评估数据
    true_predictions = [
        [tags[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    return seqeval.compute(predictions=true_predictions, references=true_labels)

def inference_demo():
    """推理演示"""
    print("加载模型进行推理演示...")
    try:
        model, tokenizer, tags = load_model("/kaggle/working/ner_ddp_model")
        model.eval()
        
        # 测试文本
        test_texts = [
            "我在北京大学学习人工智能",
            "张三在腾讯公司工作",
            "他喜欢看电影《阿甘正传》"
        ]
        
        for text in test_texts:
            print(f"\n输入文本: {text}")
            entities = predict_entities(model, tokenizer, tags, text)
            print("识别的实体:")
            for entity in entities:
                print(f"  - {entity['text']} ({entity['label']}) [{entity['start']}:{entity['end']}]")
    
    except FileNotFoundError:
        print("未找到保存的模型，请先进行训练！")

def main():
    

    ds, tags, entity_index = load_and_prepare_data()
    
 
    ds1 = process_entity_tags(ds, entity_index)
    
   
    tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
    ds2 = tokenize_data(ds1, tokenizer)
    

    model, optimizer = create_model_and_optimizer(tags)
    

    world_size = torch.cuda.device_count()
    if world_size < 2:
        print("警告: 只检测到一个GPU")
        world_size = 1
    
    print(f"使用 {world_size} 个GPU进行分布式训练...")
    ds2.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    
    if world_size > 1:
        mp.spawn(train_ddp, args=(world_size, ds2['train'], tags), nprocs=world_size, join=True)
    else:
        # 单GPU情况下直接训练
        train_ddp(0, 1, ds2['train'], tags)

    print("训练完成！")

    inference_demo()


if __name__ == "__main__":
    main()

Writing ddp.py


In [ ]:
! python ddp.py

2025-07-12 16:48:50.719703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752338930.752033     141 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752338930.759022     141 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
README.md: 100%|██████████████████████████████| 21.0/21.0 [00:00<00:00, 177kB/s]
dataset_infos.json: 100%|██████████████████████| 970/970 [00:00<00:00, 8.67MB/s]
(…)-00000-of-00001-a33d0e4276aef9b4.parquet: 100%|█| 1.30M/1.30M [00:00<00:00, 3
(…)-00000-of-00001-07f476b71c5edde6.parquet: 100%|█| 178k/178k [00:00<00:00, 92.
Generating train split: 100%|██| 10748/10748 [00:00<00:00, 248962.46 examples/s]
Generating validation split: 100